In [1]:
"""
scraping.ipynb
navitimeからバス停情報並びに、バスのダイヤデータを取得する。
"""
import pandas as pd
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import os
import pickle
import uuid
import time

from pathlib import Path

import geopandas as gpd

from fuzzywuzzy import process

c:\Users\tora2\anaconda3\envs\geo_env\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
"""
当該バス会社の全バス停情報をnavitimeから取得するプログラム
"""

"""INPUT"""
BusCompanyPage = "https://www.navitime.co.jp/bus/company/00001313/"



# 結果格納用辞書 key:nameOfBusStop value:link
bsInfoAndLinks = []
# 平仮名とリンク
kana_link_dict = {}


# バス停の平仮名からスクレイピング
response = requests.get(BusCompanyPage)
soup = BeautifulSoup(response.text, "html.parser")
links = soup.find_all("dl", class_="kana-frame")

for link in links:
    soup_c = BeautifulSoup(str(link), "html.parser")

    for a in soup_c.select("a"):
        kana_link_dict[a.text] = a["href"]

# バス停名とリンクを取得
for kana_link in list(kana_link_dict.values()):        
    response = requests.get("https://" + kana_link[2:])
    soup = BeautifulSoup(response.text,"html.parser")
    node_contents = soup.find("ul",class_="node-list")
    nodes = BeautifulSoup(str(node_contents),"html.parser").find_all("li")
    for node in nodes:
        soup_c = BeautifulSoup(str(node),"html.parser")
        url = soup_c.find("a")["href"]
        nameBusStop = soup_c.find("rb").text
        id = url.split("/")[-2]
        #結果格納
        bsInfoAndLinks.append({"nameBusStop":nameBusStop,"url":url,"id":id})

In [3]:
pd.DataFrame(bsInfoAndLinks)

,nameBusStop,url,id
0,明石(石垣市),//www.navitime.co.jp/bus/diagram/direction/004...,00428229
1,荒川(石垣市),//www.navitime.co.jp/bus/diagram/direction/004...,00428172
2,新川公園,//www.navitime.co.jp/bus/diagram/direction/004...,00428163
3,新川五町内,//www.navitime.co.jp/bus/diagram/direction/004...,00428222
4,新川小学校前,//www.navitime.co.jp/bus/diagram/direction/004...,00428199
...,...,...,...
128,吉原(石垣市),//www.navitime.co.jp/bus/diagram/direction/004...,00428170
129,米原(石垣市),//www.navitime.co.jp/bus/diagram/direction/004...,00428173
130,米原キャンプ場,//www.navitime.co.jp/bus/diagram/direction/004...,00428232
131,ヨーン,//www.navitime.co.jp/bus/diagram/direction/004...,00428182
